**Setup**

In [1]:
import os

from dotenv import load_dotenv
from openai import OpenAI
import tiktoken

# 1) Load environment variables from .env (OPENAI_API_KEY)
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError(
        "OPENAI_API_KEY is not set. "
        "Create a .env file in the project root and put your API key there."
    )

# 2) Initialize the OpenAI client (it will read OPENAI_API_KEY automatically)
client = OpenAI()

# 3) Choose the model
MODEL_NAME = "gpt-4.1-mini"

# 4) Prepare tokenizer for local token counting
encoding = tiktoken.get_encoding("o200k_base")


**LLM**

In [2]:
prompt = (
    "Explain what a Large Language Model (LLM) is for an engineering student. "
    "Use 4–5 sentences. Use clear language, but be technically correct."
)

response = client.responses.create(
    model=MODEL_NAME,
    input=prompt,
    max_output_tokens=250,
)

print("=== MODEL ANSWER ===\n")
print(response.output_text)

print("\n=== TOKEN USAGE (FROM API) ===")
print("Input tokens :", response.usage.input_tokens)
print("Output tokens:", response.usage.output_tokens)
print("Total tokens :", response.usage.total_tokens)

=== MODEL ANSWER ===

A Large Language Model (LLM) is a type of artificial intelligence designed to understand and generate human-like text by analyzing vast amounts of written data. It uses a neural network with many layers, called a transformer, which processes words in context to predict the next word in a sentence. Engineers often train these models on diverse text sources so they can perform tasks like translation, summarization, or answering questions. The "large" part means the model has millions or billions of parameters, enabling it to capture complex patterns in language. In essence, an LLM is a sophisticated pattern recognition system tailored for natural language processing.

=== TOKEN USAGE (FROM API) ===
Input tokens : 39
Output tokens: 124
Total tokens : 163


**Local token counting (tiktoken)**

In [3]:
def count_tokens(text: str) -> int:
    """
    Count tokens in a given text using the tiktoken encoding.
    """
    if not text:
        return 0
    return len(encoding.encode(text))


sample_text = (
    "Cubic Engineering Consultancy provides end-to-end design and "
    "supervision services across multiple regions in the UAE."
)

print("=== LOCAL TOKEN COUNT EXAMPLE ===")
print("Text       :", sample_text)
print("Token count:", count_tokens(sample_text))

=== LOCAL TOKEN COUNT EXAMPLE ===
Text       : Cubic Engineering Consultancy provides end-to-end design and supervision services across multiple regions in the UAE.
Token count: 19


**Cost estimation example**

In [4]:
INPUT_PRICE_PER_MILLION = 0.40
OUTPUT_PRICE_PER_MILLION = 1.60


def estimate_cost_usd(input_tokens: int, output_tokens: int) -> float:
    """
    Estimate cost of a single LLM call, in USD.
    """
    input_cost = (input_tokens / 1_000_000) * INPUT_PRICE_PER_MILLION
    output_cost = (output_tokens / 1_000_000) * OUTPUT_PRICE_PER_MILLION
    return input_cost + output_cost


input_tokens = response.usage.input_tokens
output_tokens = response.usage.output_tokens

estimated_cost = estimate_cost_usd(input_tokens, output_tokens)

print("=== COST ESTIMATE FOR THIS CALL (EXAMPLE) ===")
print("Input tokens :", input_tokens)
print("Output tokens:", output_tokens)
print(f"Estimated cost: ${estimated_cost:.8f} USD")

=== COST ESTIMATE FOR THIS CALL (EXAMPLE) ===
Input tokens : 39
Output tokens: 124
Estimated cost: $0.00021400 USD
